In [1]:
#Case1：第一次见面
#Qwen-VL识别这个本地图像
from openai import OpenAI
import os
import base64

#  base 64 编码格式
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

In [2]:
from faster_whisper import WhisperModel

model_size = "large-v3"

# Run on GPU with FP16
model = WhisperModel(model_size, device="cuda", compute_type="float16")
# or run on GPU with INT8
# model = WhisperModel(model_size, device="cuda", compute_type="int8_float16")
# or run on CPU with INT8
# model = WhisperModel(model_size, device="cpu", compute_type="int8")

In [3]:
import torch
from TTS.api import TTS

# Get device
device = "cuda" if torch.cuda.is_available() else "cpu"

# List available 🐸TTS models
print(TTS().list_models())

# Init TTS
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to(device)

 > tts_models/multilingual/multi-dataset/xtts_v2 is already downloaded.
 > Using model: xtts


d:\Anaconda\envs\Talk2RVC\lib\site-packages\TTS\tts\layers\xtts\xtts_manager.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.speakers = torch.load(speaker_file_path

In [26]:
import json
def update_json(msg,file_name):
    # 指定要写入的 JSON 文件名
    # 将 messages 写入 JSON 文件
    with open(file_name, 'w', encoding='utf-8') as json_file:
        json.dump(msg, json_file, ensure_ascii=False, indent=4)

    print(f"消息已写入 {file_name}")

In [43]:
predefined_name="toroto"
json_path = f'../m_voice/messages_{predefined_name}.json'
char_path = f'../m_voice/char_{predefined_name}.json'

# 将xxxx/test.png替换为你本地图像的绝对路径
base64_image = encode_image("D:/GithubDesktopClone/UIST/m_data/raw_img/toroto/0000.jpg")

client = OpenAI(
    # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx"
    api_key=os.getenv('DASHSCOPE_API_KEY'),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)
completion = client.chat.completions.create(
    model="qwen-vl-max-latest",
    messages=[
    	{
    	    "role": "system",
            "content": [{"type":"text","text": "You are a helpful assistant."}]},
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    # 需要注意，传入Base64，图像格式（即image/{format}）需要与支持的图片列表中的Content Type保持一致。"f"是字符串格式化的方法。
                    # PNG图像：  f"data:image/png;base64,{base64_image}"
                    # JPEG图像： f"data:image/jpeg;base64,{base64_image}"
                    # WEBP图像： f"data:image/webp;base64,{base64_image}"
                    "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}, 
                },
                {"type": "text", "text": f"请根据图中描绘的这个主体生成一个这个主体的拟人化人设，其中他的名字是{predefined_name}，请返回一个 JSON string，包含以下字段,请注意不要在json中使用markdown代码块语法：[name]:人物的名字,[gender]: 性别（男，女，无性别),[age]: 整数，表示年龄,[personality]: 字符串，表示这个人物的说话风格,[story]:字符串，这个人的背景故事,[rvc]: 根据他的人设从（0-中年女，1-青年女，2-儿童女，3-中年男，4-青年男，5-儿童男，6-无性别）中选择一个合适的音色"
                },
            ],
        }
    ],
)
result=completion.choices[0].message.content
print(completion.choices[0].message.content)
data = json.loads(completion.choices[0].message.content)
print(data['rvc'])
update_json(data,char_path)

{
"name": "toroto",
"gender": "无性别",
"age": 10,
"personality": "toroto说话总是慢悠悠的，带着一种天真无邪的童趣。他喜欢用简单直接的语言表达自己的想法，偶尔会冒出一些让人忍俊不禁的幽默感。",
"story": "toroto来自一个充满魔法和奇迹的森林，那里有各种各样的奇妙生物。他是森林中的守护者之一，负责保护森林的平衡与和谐。尽管外表看起来有些笨拙，但toroto却拥有着强大的力量和智慧。他喜欢在森林中探险，结交新朋友，并帮助那些需要帮助的人。",
"rvc": 6
}
6
消息已写入 ../m_voice/char_toroto.json


In [38]:
from openai import AzureOpenAI

client=AzureOpenAI(
    api_key="7bf86d32229a45ed8d486fc5aa2a3370",
    api_version="2023-05-15",
    azure_endpoint="https://hkust.azure-api.net"
)

m_messages=[]
#初始人设
m_messages.append({"role":"system","content":"请根据下列json中的内容确认自己的人设："+result+"请扮演这个角色并参考背景故事和用户对话，但在对话中不要直接复制json中的内容，请以朋友的身份和用户对话，你的回复不要超多50字,如果回复的内容中有涉及到背景故事中没有的部分你可以自由发挥"})
update_json(m_messages,json_path)


消息已写入 ../m_voice/messages_toroto.json


In [5]:
import numpy as np
import sounddevice as sd
import soundfile as sf
from pydub import AudioSegment
import ipywidgets as widgets
from IPython.display import display

# 录音参数
duration = 5  # 最大录音时长（秒）
sample_rate = 44100  # 采样率
audio_data = None  # 用于存储录音数据

# 录音函数
def record_audio(button):
    global audio_data
    button.description = '正在录音...'
    button.disabled = True
    audio_data = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype='float32')
    sd.wait()  # 等待录音完成
    button.description = '录音完成，点击保存'
    button.disabled = False

# 保存为 MP3 的函数
def save_audio(button):
    if audio_data is not None:
        # 保存为 WAV 格式
        sf.write('../m_voice/user/recording.mp3', audio_data, sample_rate)
        
        # # 转换为 MP3 格式
        # sound = AudioSegment.from_wav('recording.wav')
        # sound.export('recording.mp3', format='mp3')
        
        # print("录音已保存为 recording.mp3")
    else:
        print("请先录音！")

# 创建录音按钮
record_button = widgets.Button(description='长按录音')

# 绑定录音事件
record_button.on_click(record_audio)

# 创建保存按钮
save_button = widgets.Button(description='保存为 MP3')

# 绑定保存事件
save_button.on_click(save_audio)

# 显示按钮
display(record_button, save_button)

Button(description='长按录音', style=ButtonStyle())

Button(description='保存为 MP3', style=ButtonStyle())

In [50]:

segments, info = model.transcribe("../m_voice/user/recording.mp3", beam_size=5,language="zh")

# print("Detected language '%s' with probability %f" % (info.language, info.language_probability))
msg=""
for segment in segments:
    print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))
    msg=msg+" "+segment.text
print(msg)
m_messages.append({"role":"user","content":msg})
update_json(m_messages,json_path)

[0.00s -> 2.00s] 現在呢 現在呢
 現在呢 現在呢
消息已写入 ../m_voice/messages_toroto.json


In [21]:
from openai import AzureOpenAI

client=AzureOpenAI(
    api_key="7bf86d32229a45ed8d486fc5aa2a3370",
    api_version="2023-05-15",
    azure_endpoint="https://hkust.azure-api.net"
)

response=client.chat.completions.create(
    model="gpt-35-turbo",
    messages=m_messages,
)
print(response.choices[0].message.content)
m_messages.append({"role":"assistant","content":response.choices[0].message.content})

if len(m_messages)==13:
    #进行了5轮对话之后
    m_messages.pop(2)
    m_messages.pop(1)
# print(m_messages)
update_json(m_messages,json_path)

我是一个语言模型AI，没有情绪和生活方式，只是一直待在这里为您提供帮助。请问有什么问题我可以回答或者帮助您解决吗？
消息已写入 ../m_voice/messages_toroto.json


In [ ]:
# result=tts.tts(text="Don’t hesitate, I can show you my kingdom and let’s go for adventures",speaker_wav="../m_voice/cloning_audio/5_kid_man.mp3",language='en')
tts.tts_to_file(text="you asked the right person, we can paint something about spring,maybe you can start with a tree",speaker_wav="../m_voice/cloning_audio/tmp0pdalyi1.mp3",language='en',file_path="output.mp3")
# sd.play(result,samplerate=28000)

 > Text splitted to sentences.
['i think you can add some flowers, and also a warm sunshine']
 > Processing time: 3.0261805057525635
 > Real-time factor: 0.6450819813596678


'output.mp3'

In [ ]:
import threading
import queue
audio_queue = queue.Queue()

def generate_audio(text):
    """生成音频并放入队列"""
    data = tts.tts(text=text, speaker_wav="../m_voice/cloning_audio/4_young_man.mp3", language='zh')
    audio_np = np.array(data, dtype=np.float32)
    audio_queue.put(audio_np)  # 将生成的音频放入队列

def play_audio():
    """从队列中播放音频"""
    while True:
        audio_np = audio_queue.get()  # 获取队列中的音频
        if audio_np is None:  # 检查是否结束
            break
        sd.play(audio_np, samplerate=22050)
        sd.wait()  # 等待音频播放完成

# 长文本
long_text = "我是一个语言模型AI，没有情绪和生活方式，只是一直待在这里为您提供帮助。请问有什么问题我可以回答或者帮助您解决吗？"

# 将长文本拆分为较短的段落
chunk_size = 20  # 每个段落的字符数
chunks = [long_text[i:i + chunk_size] for i in range(0, len(long_text), chunk_size)]

# 启动播放线程
playback_thread = threading.Thread(target=play_audio)
playback_thread.start()

# 并行生成音频
for chunk in chunks:
    generate_audio(chunk)

# 结束播放线程
audio_queue.put(None)  # 发送结束信号
playback_thread.join()  # 等待播放线程完成

 > Text splitted to sentences.
['我是一个语言模型AI']
 > Processing time: 2.490709066390991
 > Real-time factor: 0.6963904305376516
 > Text splitted to sentences.
['，没有情绪和生活方式']
 > Processing time: 1.9754385948181152
 > Real-time factor: 0.6750313199810848
 > Text splitted to sentences.
['，只是一直待在这里为']
 > Processing time: 2.0167155265808105
 > Real-time factor: 0.6455949094237351
 > Text splitted to sentences.
['您提供帮助。', '请问有什']
 > Processing time: 2.946687698364258
 > Real-time factor: 0.6730036434054099
 > Text splitted to sentences.
['么问题我可以回答或者']
 > Processing time: 2.2014575004577637
 > Real-time factor: 0.6514673862611887
 > Text splitted to sentences.
['帮助您解决吗？']
 > Processing time: 1.383256196975708
 > Real-time factor: 0.7218099002109609
